In [1]:
import numpy as np
import cv2
import pandas as pd

In [65]:
''' file name : houghlines.py
Description : This sample shows how to detect lines using Hough Transform
This is Python version of this tutorial : http://opencv.itseez.com/doc/tutorials/imgproc/imgtrans/hough_lines/hough_lines.html
Level : Beginner
Benefits : Learn to find lines in an image and draw them
Usage : python houghlines.py 
Written by : Abid K. (abidrahman2@gmail.com) , Visit opencvpython.blogspot.com for more tutorials '''


print (" Hough Lines demo ")
print (" Press h to draw lines using cv2.HoughLines()")
print (" Press p to draw lines using cv2.HoughLinesP()")
print (" All the parameter values selected at random, Change it the way you like")

im = cv2.imread('/mnt/c/Users/Михаил/Pictures/skyscraper2.jpg')
gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,150,200,apertureSize = 3, L2gradient=True)

#cv2.imshow('houghlines',im)
cv2.imwrite('/mnt/c/Users/Михаил/Pictures/gray.jpg', gray)
cv2.imwrite('/mnt/c/Users/Михаил/Pictures/edges.jpg', edges)

  

 Hough Lines demo 
 Press h to draw lines using cv2.HoughLines()
 Press p to draw lines using cv2.HoughLinesP()
 All the parameter values selected at random, Change it the way you like


True

In [44]:
help(cv2.Canny)

Help on built-in function Canny:

Canny(...)
    Canny(image, threshold1, threshold2[, edges[, apertureSize[, L2gradient]]]) -> edges
    .   @brief Finds edges in an image using the Canny algorithm @cite Canny86 .
    .   
    .   The function finds edges in the input image and marks them in the output map edges using the
    .   Canny algorithm. The smallest value between threshold1 and threshold2 is used for edge linking. The
    .   largest value is used to find initial segments of strong edges. See
    .   <http://en.wikipedia.org/wiki/Canny_edge_detector>
    .   
    .   @param image 8-bit input image.
    .   @param edges output edge map; single channels 8-bit image, which has the same size as image .
    .   @param threshold1 first threshold for the hysteresis procedure.
    .   @param threshold2 second threshold for the hysteresis procedure.
    .   @param apertureSize aperture size for the Sobel operator.
    .   @param L2gradient a flag, indicating whether a more accurate \

In [13]:
help(cv2.HoughLines)

Help on built-in function HoughLines:

HoughLines(...)
    HoughLines(image, rho, theta, threshold[, lines[, srn[, stn[, min_theta[, max_theta]]]]]) -> lines
    .   @brief Finds lines in a binary image using the standard Hough transform.
    .   
    .   The function implements the standard or standard multi-scale Hough transform algorithm for line
    .   detection. See <http://homepages.inf.ed.ac.uk/rbf/HIPR2/hough.htm> for a good explanation of Hough
    .   transform.
    .   
    .   @param image 8-bit, single-channel binary source image. The image may be modified by the function.
    .   @param lines Output vector of lines. Each line is represented by a two-element vector
    .   \f$(\rho, \theta)\f$ . \f$\rho\f$ is the distance from the coordinate origin \f$(0,0)\f$ (top-left corner of
    .   the image). \f$\theta\f$ is the line rotation angle in radians (
    .   \f$0 \sim \textrm{vertical line}, \pi/2 \sim \textrm{horizontal line}\f$ ).
    .   @param rho Distance resolution

In [81]:
img = im.copy()
lines = cv2.HoughLines(edges,1,np.pi/180,130)
#print (lines)
for line in lines:
    for rho,theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))   # Here i have used int() instead of rounding the decimal value, so 3.8 --> 3
        y1 = int(y0 + 1000*(a))    # But if you want to round the number, then use np.around() function, then 3.8 --> 4.0
        x2 = int(x0 - 1000*(-b))   # But we need integers, so use int() function after that, ie int(np.around(x))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
cv2.imwrite('/mnt/c/Users/Михаил/Pictures/houghlines.jpg', img)
#cv2.imshow('houghlines',img)

True

In [67]:
img = im.copy()
lines = cv2.HoughLinesP(edges,1,np.pi/180,150, minLineLength = 50, maxLineGap = 200)
for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
cv2.imwrite('/mnt/c/Users/Михаил/Pictures/houghlines_p.jpg', img)
#cv2.imshow('houghlines',img)


True